
# Getting Started with LiveKit Agents: A Jupyter Notebook Tutorial

Welcome to this tutorial on LiveKit Agents! In this notebook, we'll walk you through the process of building your first AI-powered voice agent. We'll cover the essential concepts and provide you with a hands-on example to get you started.

## 1. Introduction to LiveKit Agents

LiveKit Agents is a powerful framework for building real-time, multimodal AI agents. These agents can interact with users through voice, video, and text, making them ideal for a wide range of applications, from customer support bots to interactive games. 

At its core, a LiveKit Agent is a stateful, long-running process that connects to the LiveKit network using WebRTC. This allows for low-latency communication, which is crucial for creating natural and engaging user experiences.

## 2. Prerequisites

Before we begin, make sure you have the following:

- **Python 3.9+** installed on your system.
- A **LiveKit Cloud account** (you can sign up for a free account at [cloud.livekit.io](https://cloud.livekit.io)).
- **API keys** for the services you want to use (e.g., OpenAI for LLM and TTS, Deepgram for STT). For this tutorial we will use OpenAI for all three.
- The **LiveKit CLI** installed. You can install it with `brew install livekit-cli` on macOS, `winget install LiveKit.LiveKitCLI` on Windows, or `curl -sSL https://get.livekit.io/cli | bash` on Linux.

## 3. Setup

Let's set up our project environment.

In [27]:
#%pip install livekit-agents "livekit-plugins-openai" python-dotenv -q

Create a file named `.env` in your project directory and add your API keys and LiveKit credentials to it. This file will be used to load your credentials securely.

```
# .env
LIVEKIT_URL=wss://YOUR_PROJECT_URL.livekit.cloud
LIVEKIT_API_KEY=YOUR_API_KEY
LIVEKIT_API_SECRET=YOUR_API_SECRET

OPENAI_API_KEY=YOUR_OPENAI_API_KEY
```

You can find your LiveKit credentials in your LiveKit Cloud project settings.

In [28]:
# Sanity check: environment & readiness
import os, socket, sys
print('LIVEKIT_URL =', os.getenv('LIVEKIT_URL'))
print('LIVEKIT_API_KEY set =', bool(os.getenv('LIVEKIT_API_KEY')))
print('LIVEKIT_API_SECRET set =', bool(os.getenv('LIVEKIT_API_SECRET')))
print('OPENAI_API_KEY set =', bool(os.getenv('OPENAI_API_KEY')))
# Basic DNS resolution check for LiveKit URL host
try:
    url = os.getenv('LIVEKIT_URL') or ''
    host = url.replace('wss://','').replace('ws://','').split('/')[0] if url else ''
    if host:
        socket.gethostbyname(host)
        print('DNS resolved for host:', host)
    else:
        print('No LIVEKIT_URL host to resolve')
except Exception as e:
    print('DNS resolution failed:', e)
print('Python:', sys.version)

LIVEKIT_URL = wss://datumverse-8iuxuc4c.livekit.cloud
LIVEKIT_API_KEY set = True
LIVEKIT_API_SECRET set = True
OPENAI_API_KEY set = True
DNS resolved for host: datumverse-8iuxuc4c.livekit.cloud
Python: 3.10.14 (main, May 13 2025, 23:24:13) [Clang 15.0.0 (clang-1500.3.9.4)]


## 4. Building a Simple Voice Agent

Now that our environment is set up, let's write the code for our voice agent directly in this notebook. We'll create a simple agent that:

1. Listens to the user's speech.
2. Transcribes the speech to text.
3. Uses a large language model (LLM) to generate a response.
4. Converts the text response to speech and plays it back to the user.

Run the next cell to define the agent entrypoint and helper setup.

In [32]:
import asyncio
import os
from dotenv import load_dotenv
from livekit import agents
from livekit.plugins import openai

# Load environment variables from .env file
load_dotenv()

async def entrypoint(ctx: agents.JobContext):
    """Entrypoint for the agent.

    This function is called when a new job is assigned to the agent. It sets up the
    agent's session and starts the interaction with the user.
    """
    await ctx.connect()

    # Initialize the speech-to-text (STT), large language model (LLM), and
    # text-to-speech (TTS) plugins from OpenAI.
    stt = openai.STT()
    llm = openai.LLM(model="gpt-4o")
    tts = openai.TTS(model="tts-1")

    # Create a chat context for the LLM. This will store the conversation history.
    chat_ctx = agents.llm.ChatContext()
    chat_ctx.append(role="system", text="You are a friendly voice assistant.")

    # Create a voice agent with the specified plugins and chat context.
    agent = agents.VoiceAgent(stt=stt, llm=llm, tts=tts, chat_ctx=chat_ctx)

    # Start the agent's interaction in the room.
    agent.start(ctx.room)

    # Greet the user when the agent joins the room.
    await asyncio.sleep(1)
    await agent.say("Hello! How can I help you today?")

# Helper to create a worker (will be started in a separate cell)
LK_URL = os.getenv("LIVEKIT_URL", "")
LK_KEY = os.getenv("LIVEKIT_API_KEY", "")
LK_SECRET = os.getenv("LIVEKIT_API_SECRET", "")
opts = agents.WorkerOptions(
    entrypoint_fnc=entrypoint,
    ws_url=LK_URL,
    api_key=LK_KEY,
    api_secret=LK_SECRET,
)
worker = agents.Worker(opts, devmode=True, register=True)

## 5. Running the Agent

Since we're in a notebook, we'll start the worker from a cell instead of a terminal command.

- Ensure your `.env` is set with LIVEKIT_URL, LIVEKIT_API_KEY, LIVEKIT_API_SECRET, and OPENAI_API_KEY.
- Run the next cell to start the worker in the foreground.
- Then, open the LiveKit Agents Playground at https://agents-playground.livekit.io, connect to your project, and interact with your agent.

In [33]:
# Start the worker (blocks this cell while running)
# Stop the cell (interrupt kernel) to shut down the worker.
from livekit import agents

try:
    worker.run()
except KeyboardInterrupt:
    print("Worker stopped.")

/var/folders/pv/g_b0j0n53rz5fm8yrlw3jg040000gn/T/ipykernel_66807/1005654844.py:6: RuntimeWarning: coroutine 'Worker.run' was never awaited
  worker.run()


In [34]:
# Quick check (optional): confirm worker has run()
import inspect
print('Has run:', hasattr(worker, 'run'))
if hasattr(worker, 'run'):
    try:
        print('run signature:', inspect.signature(worker.run))
    except Exception as e:
        print('Could not inspect run signature:', e)

Has run: True
run signature: () -> 'None'


## 6. Conclusion and Next Steps

Congratulations! You've successfully built and run your first LiveKit Agent. This is just the beginning of what you can do with the LiveKit Agents framework. From here, you can explore:

- **Different plugins**: Try out other STT, LLM, and TTS providers.
- **Function calling**: Enable your agent to interact with external APIs.
- **Multimodal agents**: Build agents that can understand and generate video as well as audio.

For more information and advanced examples, check out the [LiveKit Agents documentation](https://docs.livekit.io/agents/).